## Code to build likelihood ratio estimator

Estimating 

$r = \frac{P(D_W, D_P, \theta)}{P(D_W, \theta) P(D_P)}$

In [2]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import tensorflow_probability as tfp
import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
from tensionnet import wmapplanck
from cmblike.data import get_data
from tqdm import tqdm

wmapraw, lwmap = get_data(base_dir='cosmology-data/').get_wmap()
praw, l = get_data(base_dir='cosmology-data/').get_planck()


Code takes a library of CAMB cls and corresponding parameters and makes a set of observations of the power spectrum from Planck and WMAP as if they were viewing the same sky...

In [11]:
nSamples = 500
joint = wmapplanck.jointClGen()
clLibrary = np.load('cl_library/cls.npy')[:nSamples]
paramsLibrary = np.load('cl_library/params.npy')[:nSamples]

Aplanck = l*(l+1)/(2*np.pi)
Awmap = lwmap*(lwmap+1)/(2*np.pi)

wmapData = []
planckData = []
for i in tqdm(range(len(paramsLibrary))):
    params = paramsLibrary[i]
    cls = clLibrary[i]
    planck, wmap = joint([], params, clexample=cls)
    wmapData.append(wmap)
    planckData.append(planck)
wmapData = np.array(wmapData)
planckData = np.array(planckData)

splitIdx = np.arange(len(wmapData))
np.random.shuffle(splitIdx)
trainIdx = splitIdx[:int(len(wmapData)*0.8)]
testIdx = splitIdx[int(len(wmapData)*0.8):]

trainWmap = wmapData[trainIdx]
testWmap = wmapData[testIdx]
trainPlanck = planckData[trainIdx]
testPlanck = planckData[testIdx]
trainParams = paramsLibrary[trainIdx]
testParams = paramsLibrary[testIdx]

normtrainWmapData = (trainWmap -np.mean(trainWmap))/np.std(trainWmap)
normtestWmapData = (testWmap -np.mean(trainWmap))/np.std(trainWmap)
normtrainPlanckData = (trainPlanck -np.mean(trainPlanck))/np.std(trainPlanck)
normtestPlanckData = (testPlanck -np.mean(trainPlanck))/np.std(trainPlanck)
normtrainParams = (trainParams -np.mean(trainParams, axis=0))/np.std(trainParams, axis=0)
normtestParams = (testParams -np.mean(trainParams, axis=0))/np.std(trainParams, axis=0)


matchedtrainData = np.hstack([normtrainWmapData, normtrainParams, normtrainPlanckData])
matchedtrainLabels = np.ones(len(matchedtrainData))
matchedtestData = np.hstack([normtestWmapData, normtestParams, normtestPlanckData])
matchedtestLabels = np.ones(len(matchedtestData))

idx = np.arange(len(matchedtrainData))
np.random.shuffle(idx)
shuffledtrainPlanck = normtrainPlanckData[idx]
shuffledtrainData = np.hstack([normtrainWmapData, normtrainParams, shuffledtrainPlanck])
shuffledtrainLabels = np.zeros(len(shuffledtrainData))

data_train = np.vstack([matchedtrainData, shuffledtrainData])
labels_train = np.hstack([matchedtrainLabels, shuffledtrainLabels])

idx = np.arange(len(matchedtestData))
np.random.shuffle(idx)
shuffledtestPlanck = normtestPlanckData[idx]
shuffledtestData = np.hstack([normtestWmapData, normtestParams, shuffledtestPlanck])
shuffledtestLabels = np.zeros(len(shuffledtestData))

data_test = np.vstack([matchedtestData, shuffledtestData])
labels_test = np.hstack([matchedtestLabels, shuffledtestLabels])

idx = np.arange(len(data_train))
np.random.shuffle(idx)
data_train = data_train[idx]
labels_train = labels_train[idx]

idx = np.arange(len(data_test))
np.random.shuffle(idx)
data_test = data_test[idx]
labels_test = labels_test[idx]



 57%|█████▋    | 285/500 [01:36<01:12,  2.97it/s]

In [4]:
optimizer = tf.keras.optimizers.legacy.Adam(
                learning_rate=1e-3)

model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(data_train.shape[1], activation='sigmoid'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(100, activation='sigmoid',
                        kernel_initializer=tf.keras.initializers.GlorotNormal()),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(100, activation='sigmoid',
                        kernel_initializer=tf.keras.initializers.GlorotNormal()),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(1, activation='linear',
                        kernel_initializer=tf.keras.initializers.GlorotNormal()),
])

In [5]:
@tf.function(jit_compile=True)
def _test_step(param, truth):
        
    r"""
    This function is used to calculate the loss value at each epoch and
    adjust the weights and biases of the neural networks via the
    optimizer algorithm.
    """
    prediction = tf.transpose(model(param, training=True))[0]
    prediction = tf.keras.layers.Activation('sigmoid')(prediction)
    truth = tf.convert_to_tensor(truth)
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)(truth, prediction)
    return loss

@tf.function(jit_compile=True)
def _train_step(params, truth):

    r"""
    This function is used to calculate the loss value at each epoch and
    adjust the weights and biases of the neural networks via the
    optimizer algorithm.
    """

    with tf.GradientTape() as tape:
        prediction = tf.transpose(model(params, training=True))[0]
        prediction = tf.keras.layers.Activation('sigmoid')(prediction)
        truth = tf.convert_to_tensor(truth)
        loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)(truth, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(
        zip(gradients,
            model.trainable_variables))
    return loss


Training loop...

In [10]:

epochs = 50
batch_size = 300
patience = 10

train_dataset = np.hstack([data_train, labels_train[:, np.newaxis]]).astype(np.float32)
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset)
train_dataset = train_dataset.batch(batch_size)

test_dataset = np.hstack([data_test, labels_test[:, np.newaxis]]).astype(np.float32)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset)
test_dataset = test_dataset.batch(batch_size)

loss_history = []
test_loss_history = []
epoch_loss_avg = tf.keras.metrics.Mean()
c = 0
#for i in tqdm.tqdm(range(epochs)):
for i in range(epochs):

    loss = [_train_step(x[:, :-1], x[:, -1]) for x in  train_dataset]
    epoch_loss_avg.update_state(loss)
    loss_history.append(epoch_loss_avg.result())

    test_loss = [_test_step(x[:, :-1], x[:, -1]) for x in  test_dataset]
    test_loss_history.append(tf.reduce_mean(test_loss))
    print('Epoch: {} Loss: {:.5f} Test Loss: {:.5f}'.format(
        i, loss_history[-1], test_loss_history[-1]))

    c += 1
    if i == 0:
        minimum_loss = test_loss_history[-1]
        minimum_epoch = i
        minimum_model = model
    else:
        if test_loss_history[-1] < minimum_loss:
            minimum_loss = test_loss_history[-1]
            minimum_epoch = i
            minimum_model = model
            c = 0
    if minimum_model:
        if c == patience:
            print('Early stopped. Epochs used = ' + str(i) +
                    '. Minimum at epoch = ' + str(minimum_epoch))
            model = minimum_model
            break

plt.plot(loss_history, label='train')
plt.plot(test_loss_history, label='test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_history.png', dpi=300, bbox_inches='tight')


NameError: name 'data' is not defined

In [9]:
from cmblike.noise import wmap_noise
from cmblike.cmb import CMB

wmap_noise = wmap_noise(l).calculate_noise()
cmbs = CMB()
likelihood = cmbs.get_likelihood(wmap, lwmap, noise=wmap_noise)

def __call__(planck, wmap, params, nonNormParams):

        r_values = []
        for i in range(len(params)):
            ps = tf.convert_to_tensor(np.array([[*wmap, *params[i], *planck]]).astype('float32'))
            logr = model(ps).numpy()[0]
            r_values.append(logr + likelihood(nonNormParams[i])[0] + 587.5)

        r_values = np.array(r_values).T[0]
        return r_values


normWmap = (wmap - np.mean(trainWmap))/np.std(trainWmap)
normPlanck = (praw - np.mean(trainPlanck))/np.std(trainPlanck)

samples = paramsLibrary[:100]
norm_params = (samples - np.mean(trainParams, axis=0))/np.std(trainParams, axis=0)

# logL - log Z
r_values = __call__(normPlanck, normWmap, norm_params, samples)

cbar = plt.scatter(samples[:, 0], samples[:, 1], c=r_values, s=10, cmap='viridis')
plt.colorbar(cbar)
plt.xlabel('Omega_c h^2')
plt.ylabel('Omega_b h^2')


ValueError: operands could not be broadcast together with shapes (45,) (83,) 